In [7]:
import uuid, cv2, json, os
from deepface import DeepFace

In [8]:
def extract_face_information(img_path):
    face_objs = DeepFace.represent(
                                img_path = img_path,
                                model_name = "Facenet",
                                enforce_detection = False
                                )
    img_path = img_path.replace("\\", "/")
    user_name = img_path.split("/")[-2]

    if len(face_objs) >= 1:
        annotations = []
        for i in range(len(face_objs)):
            annotation = {}
            face = face_objs[i]
            facial_area = face['facial_area']
            x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']

            annotation["image_path"] = img_path
            annotation["facial_area"] = facial_area
            annotation["x"] = x
            annotation["y"] = y
            annotation["w"] = w
            annotation["h"] = h

            annotations.append(annotation)

        return annotations

    return None

def video_annotations(video_path):
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rand_name = f"{uuid.uuid4()}.jpg"
        image_path = f"data/face_annotations/images/{rand_name}"
        cv2.imwrite(image_path, frame)

        annotations = extract_face_information(image_path)

        if annotations:
            with open(f"data/face_annotations/annotations/{rand_name}.json", "w") as f:
                json.dump(annotations, f, indent=4)
        else:
            os.remove(image_path)

    cap.release()

In [9]:
for root, dirs, files in os.walk("data/face_annotations/videos"):
    for file in files:
        video_annotations(os.path.join(root, file))
# video_annotations("data/face_annotations/videos/3181891-uhd_3840_2160_25fps.mp4")